In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('Reddit_Encoded.csv')
df

,comment,hate_speech,lemmatized_comment,document_vector_flat
0,subsection retarded hungarians ohh boy brace l...,1,subsection retard hungarians ohh boy brace liv...,"0.014043219,-0.01809359,0.017145459,0.08062436..."
1,hiii just got work Foundation and grounding ma...,0,hiii just get work Foundation and ground mainl...,"-0.0030388932,-0.035133556,0.020659983,0.07383..."
2,wow guess soyboys every country,0,wow guess soyboys every country,"0.017362628,0.005587179,0.0297773,0.109146975,..."
3,owen benjamins soyboy song goes every country ...,0,owen benjamins soyboy song go every country amaze,"0.018085241,0.0011954829,2.8959475e-05,0.07601..."
4,yall hear sumn means live small town rn for w...,0,yall hear sumn mean live small town rn for wor...,"0.023993038,-0.00060867134,0.005239945,0.05989..."
...,...,...,...,...
22206,op stop faggot post videos next time hard,1,op stop faggot post videos next time hard,"0.08339707,-0.017676119,-0.036872935,0.1038081..."
22207,minute long video top hate champagne goes need...,0,minute long video top hate champagne go need g...,"0.009342635,0.02728245,-0.0013451587,0.0765217..."
22208,clue whos ecelebs are point time need get alo...,1,clue whos ecelebs be point time need get along...,"-0.0006810841,0.008315975,0.029024707,0.077029..."
22209,didn’t insult you insult me,0,didn ’ t insult you insult me,"0.06474433,-0.17082486,-0.029483724,0.04382453..."


In [6]:
df['fasttext_vec']=df['document_vector_flat'].apply(lambda x:np.fromstring(x.strip('[]'),sep=','))
X=np.vstack(df['fasttext_vec'].values)
y=df['hate_speech']

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

In [8]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority',random_state=42)
X_sm, y_sm = smote.fit_resample(X_train,y_train)
y_sm.value_counts()

0    13539
1    13539
Name: hate_speech, dtype: int64

In [9]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_score,recall_score,f1_score
from sklearn.model_selection import cross_val_score

In [12]:
from sklearn.svm import SVC

In [13]:
# Create a non-linear kernel SVM classifier
svm = SVC(kernel='poly',degree=5, random_state=42)
svm.fit(X_sm, y_sm)

SVC(degree=5, kernel='poly', random_state=42)

In [16]:
y_pred_svm=svm.predict(X_test)

In [17]:
print("Support Vector Machine Performance:")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))

Support Vector Machine Performance:
Confusion Matrix:
 [[3009  377]
 [ 400  657]]
              precision    recall  f1-score   support

           0       0.88      0.89      0.89      3386
           1       0.64      0.62      0.63      1057

    accuracy                           0.83      4443
   macro avg       0.76      0.76      0.76      4443
weighted avg       0.82      0.83      0.82      4443



In [19]:
accuracy=accuracy_score(y_test,y_pred_svm)
precision=precision_score(y_test,y_pred_svm)
recall=recall_score(y_test,y_pred_svm)
f1=f1_score(y_test,y_pred_svm)
print(f"Accuracy:{accuracy:.2f}")
print(f"precision:{precision:.2f}")
print(f"recall:{recall:.2f}")
print(f"f1:{f1:.2f}")

Accuracy:0.83
precision:0.64
recall:0.62
f1:0.63


In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV 

In [4]:
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001,'scale'], 
              'kernel': ['rbf','linear','poly']}  

In [23]:
svm_r=SVC()

In [24]:
random_search = RandomizedSearchCV(svm_r,
                                   param_grid) 
random_search.fit(X_train, y_train) 
print(random_search.best_estimator_)

SVC(C=10)


In [25]:
print(random_search.best_params_)

{'kernel': 'rbf', 'gamma': 'scale', 'C': 10}


In [29]:
svm = SVC(C=10,kernel='rbf',degree=5,gamma='scale',random_state=42)

In [30]:
svm.fit(X_sm, y_sm)

SVC(C=10, degree=5, random_state=42)

In [31]:
y_pred_svm=svm.predict(X_test)

In [32]:
print("Support Vector Machine Performance:")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))

Support Vector Machine Performance:
Confusion Matrix:
 [[2924  462]
 [ 276  781]]
              precision    recall  f1-score   support

           0       0.91      0.86      0.89      3386
           1       0.63      0.74      0.68      1057

    accuracy                           0.83      4443
   macro avg       0.77      0.80      0.78      4443
weighted avg       0.85      0.83      0.84      4443



In [33]:
accuracy=accuracy_score(y_test,y_pred_svm)
precision=precision_score(y_test,y_pred_svm)
recall=recall_score(y_test,y_pred_svm)
f1=f1_score(y_test,y_pred_svm)
print(f"Accuracy:{accuracy:.2f}")
print(f"precision:{precision:.2f}")
print(f"recall:{recall:.2f}")
print(f"f1:{f1:.2f}")

Accuracy:0.83
precision:0.63
recall:0.74
f1:0.68
